In [3]:
#importação de bibliotecas
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# 1. Carregar o dataset balanceado

In [4]:
df = pd.read_csv('dataset_pronto_treino.csv')

# 2. Separar Features (X) e Alvo (y)
X = df[['temp', 'umidade']].values
y = df['classe'].values

# 3. Divisão de Treino e Teste (80% / 20%)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.preprocessing import StandardScaler
scaler_treino = StandardScaler()
X_train_scaled = scaler_treino.fit_transform(X_train)
X_test_scaled = scaler_treino.transform(X_test)

# 4. Arquitetura da Rede Neural (TinyML)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(2,)), # Entrada: Temp e Umidade
    tf.keras.layers.Dense(8, activation='relu'),                  # Camada Oculta
    tf.keras.layers.Dense(3, activation='softmax')                # Saída: 3 Classes (Probabilidades)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# 5. Treinamento

In [7]:
print("Iniciando o treinamento...")
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=16,
                    validation_split=0.1, verbose=1)

Iniciando o treinamento...
Epoch 1/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5984 - loss: 0.8766 - val_accuracy: 0.6592 - val_loss: 0.5125
Epoch 2/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7186 - loss: 0.4459 - val_accuracy: 0.9435 - val_loss: 0.3673
Epoch 3/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9619 - loss: 0.3174 - val_accuracy: 0.9777 - val_loss: 0.2142
Epoch 4/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9796 - loss: 0.1765 - val_accuracy: 0.9846 - val_loss: 0.1162
Epoch 5/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9841 - loss: 0.0993 - val_accuracy: 0.9812 - val_loss: 0.0846
Epoch 6/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9811 - loss: 0.0771 - val_accuracy: 0.9863 - val_loss: 0.0727
Epoch 7/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9822 - loss: 0.0666 - val_accuracy: 0.9846 - val_loss: 0.0671
Epoch 8/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.982

# 6. Avaliação

In [8]:
loss, acc = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"\n✅ Acurácia final nos dados de teste: {acc*100:.2f}%")


✅ Acurácia final nos dados de teste: 98.97%


# 7. Conversão para TensorFlow Lite (Formato para microcontrolador)

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Salvar o arquivo binário
with open('modelo_predator.tflite', 'wb') as f:
    f.write(tflite_model)

print("\n🚀 Arquivo 'modelo_predator.tflite' gerado com sucesso!")

Saved artifact at '/tmp/tmpx1jt600l'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 2), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  133216957660816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133216957661008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133216957659280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133216952074320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133216957659856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133216957659472: TensorSpec(shape=(), dtype=tf.resource, name=None)

🚀 Arquivo 'modelo_predator.tflite' gerado com sucesso!


#8.Geração do Arquivo de Cabeçalho C

In [10]:
import os

def convert_to_c_array(tflite_path, model_name="modelo_predator"):
    with open(tflite_path, 'rb') as f:
        tflite_content = f.read()

    hex_lines = [f'0x{byte:02x}' for byte in tflite_content]

    c_str = f'#ifndef {model_name.upper()}_H\n'
    c_str += f'#define {model_name.upper()}_H\n\n'
    c_str += f'// Tamanho do modelo: {len(tflite_content)} bytes\n'
    c_str += f'const unsigned int {model_name}_len = {len(tflite_content)};\n\n'
    c_str += f'const unsigned char {model_name}[] __attribute__((aligned(16))) = {{\n'

    for i in range(0, len(hex_lines), 12):
        c_str += '  ' + ', '.join(hex_lines[i:i+12]) + ',\n'

    c_str += '};\n\n#endif\n'

    with open(f"{model_name}.h", 'w') as f:
        f.write(c_str)
    return f"{model_name}.h"

header_file = convert_to_c_array('modelo_predator.tflite')
print(f"🔥 Arquivo '{header_file}' pronto para download na pasta lateral do Colab!")

🔥 Arquivo 'modelo_predator.h' pronto para download na pasta lateral do Colab!
